In [ ]:
import keras
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess data
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

def build_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

# Perform k-fold cross-validation
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f'processing fold #{i}')
    # Prepare the validation data
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)

    # Build the Keras model
    model = build_model(train_data.shape[1:])

    # Train the model
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)

    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

print(f'Mean MAE: {np.mean(all_scores)}')

# Train the network for a longer period (500 epochs)
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print(f'processing fold #{i}')
    # Prepare the validation data
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)

    # Build the Keras model
    model = build_model(train_data.shape[1:])

    # Train the model
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, validation_data=(val_data, val_targets), verbose=0)
    mae_history = history.history['val_mae']  # Corrected key here
    all_mae_histories.append(mae_history)

# Compute the average of the per-epoch MAE scores for all folds
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

# Smooth the curve
def smooth_curve(points, factor=0.9):
    smoothed_points = [points[0]]
    for point in points[1:]:
        smoothed_points.append(smoothed_points[-1] * factor + point * (1 - factor))
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

processing fold #0
processing fold #1
processing fold #2
